In [147]:
import pandas as pd
import numpy as np
import sys

## Replace id with the OSC ID

In [184]:
filesuffix = !curl -s "http://openstreetcam.org/details" --data "id=1249727" | jq .[] | grep meta | awk -F\" '{print $4}'

## Download the sensor reading file associated with the OSC ID

In [185]:
filesuffix = ''.join(filesuffix)
filename='www.openstreetcam.org/'+ filesuffix
!wget $filename

--2018-07-29 20:49:46--  http://www.openstreetcam.org/storage8/files/photo/2018/6/15/1249727_5b2306ecd0e60.txt
Resolving www.openstreetcam.org (www.openstreetcam.org)... 78.47.202.71
Connecting to www.openstreetcam.org (www.openstreetcam.org)|78.47.202.71|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://storage8.openstreetcam.org/files/photo/2018/6/15/1249727_5b2306ecd0e60.txt [following]
--2018-07-29 20:49:47--  http://storage8.openstreetcam.org/files/photo/2018/6/15/1249727_5b2306ecd0e60.txt
Resolving storage8.openstreetcam.org (storage8.openstreetcam.org)... 94.130.205.166
Connecting to storage8.openstreetcam.org (storage8.openstreetcam.org)|94.130.205.166|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2616505 (2.5M) [text/plain]
Saving to: ‘1249727_5b2306ecd0e60.txt’

1249727_5b2306ecd0e 100%[===================>]   2.50M   637KB/s    in 4.0s    

2018-07-29 20:49:51 (637 KB/s) - ‘1249727_5b2306ecd0e60.txt’ s

The general shape of the OSC metadata_file file is as follows:
```
accel_reading1
accel_reading2
accel_reading3
accel_reading4
accel_reading5
accel_reading6
accel_reading7
accel_reading8
accel_reading9
accel_reading10
location_reading
```
The code below: 
- rolls up all the `accel_reading_*` values and calculates the `average(magnitude(accel_reading*_x,accel_reading*_y,accel_reading*_z))`
- Assigns the "bumpiness score" to the location_reading.

The resultant shape is:
# `location_reading<>rolledUp_accelMagnitude`

## Replace `read_csv` with the file you just downloaded and `to_csv` to `OSC_id-result.csv`

In [186]:
# Refer https://github.com/openstreetcam/openstreetview.org/issues/109 for file spec
df = pd.read_csv("1249727_5b2306ecd0e60.txt",header=None,skiprows=1,delimiter=";",names=["timestamp","longitude","latitude","elevation","horizontal_accuracy","GPSspeed","yaw","pitch","roll","accelerationX","accelerationY","accelerationZ","pressure","compass","videoIndex","tripFrameIndex","gravityX","gravityY","gravityZ","OBD2speed","vertical_accuracy"])
df['bump']=0
locs =  df.loc[df['longitude'].notnull()].index
bump=[]

for idx,i in enumerate(locs):
    bump_t=[]
    try:
        if idx == 0:
            next_val = locs[0]
            for idx2 in xrange(0,i):
                a_x = df['accelerationX'][idx2]
                a_y = df['accelerationY'][idx2]
                a_z = df['accelerationZ'][idx2]
                bump_val = np.sqrt(a_x**2 + a_y**2 + a_z**2)
                bump_t.append(bump_val)
            bump_avg = np.nansum(bump_t) / float(np.count_nonzero(~np.isnan(bump_t)))
            #print "Assining ",bump_avg," to row",i
            df.loc[i,'bump'] = bump_avg
        else:
            next_val = locs[idx + 1]
    except IndexError:
        print "end"
        break
    for idx2 in xrange(i,next_val):
        a_x = df['accelerationX'][idx2]
        a_y = df['accelerationY'][idx2]
        a_z = df['accelerationZ'][idx2]
        bump_val = np.sqrt(a_x**2 + a_y**2 + a_z**2)
        bump_t.append(bump_val)
    bump_avg = np.nansum(bump_t) / float(np.count_nonzero(~np.isnan(bump_t)))
    #print "Assining ",bump_avg," to row",i
    df.loc[i,'bump'] = bump_avg
    #print "Avg of all the accels for ",idx, bump_avg
df.iloc[locs].to_csv('1249727-parsed.csv')

/Users/vadibhatla/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/Users/vadibhatla/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


end


## The last column contains the Bumpiness Score and Location Readings

In [188]:
df.iloc[locs].head(100)

,timestamp,longitude,latitude,elevation,horizontal_accuracy,GPSspeed,yaw,pitch,roll,accelerationX,...,pressure,compass,videoIndex,tripFrameIndex,gravityX,gravityY,gravityZ,OBD2speed,vertical_accuracy,bump
0,1.529019e+09,-105.156575,39.987288,1715.1,18.1,12.119488,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,31.0,NaN
1,1.529019e+09,-105.156575,39.987288,1715.1,18.1,12.119488,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,0.088868
6,1.529019e+09,-105.156582,39.987244,1713.4,18.1,17.645857,NaN,NaN,NaN,NaN,...,NaN,186.0,NaN,NaN,NaN,NaN,NaN,NaN,31.0,0.111884
17,1.529019e+09,-105.156586,39.987200,1712.5,18.1,18.577412,NaN,NaN,NaN,NaN,...,NaN,183.0,NaN,NaN,NaN,NaN,NaN,NaN,31.0,NaN
18,1.529019e+09,-105.156586,39.987200,1712.5,18.1,18.577412,NaN,NaN,NaN,NaN,...,NaN,183.0,1.0,1.0,NaN,NaN,NaN,NaN,31.0,0.125381
29,1.529019e+09,-105.156576,39.987144,1712.4,18.1,15.277148,NaN,NaN,NaN,NaN,...,NaN,172.0,2.0,2.0,NaN,NaN,NaN,NaN,31.0,NaN
30,1.529019e+09,-105.156576,39.987144,1712.4,18.1,15.277148,NaN,NaN,NaN,NaN,...,NaN,172.0,NaN,NaN,NaN,NaN,NaN,NaN,31.0,0.170510
41,1.529019e+09,-105.156567,39.987115,1713.6,18.1,9.782264,NaN,NaN,NaN,NaN,...,NaN,165.0,NaN,NaN,NaN,NaN,NaN,NaN,30.9,NaN
42,1.529019e+09,-105.156567,39.987115,1713.6,18.1,9.782264,NaN,NaN,NaN,NaN,...,NaN,165.0,3.0,3.0,NaN,NaN,NaN,NaN,30.9,0.154728
53,1.529019e+09,-105.156570,39.987102,1713.8,18.0,5.320796,NaN,NaN,NaN,NaN,...,NaN,190.0,NaN,NaN,NaN,NaN,NaN,NaN,30.9,NaN
